In [1]:
import ee

In [2]:
#ee.Authenticate()

In [3]:
ee.Initialize()

In [3]:

import geemap

Map = geemap.Map()
import geemap

Map = geemap.Map()

TempMin = ee.ImageCollection("users/cnav/RediamClimate/TempMinMonth")
TempMax = ee.ImageCollection("users/cnav/RediamClimate/TempMaxMonth")
TempMean = ee.ImageCollection("users/cnav/RediamClimate/TempMeanMonth")
Andalucia = ee.FeatureCollection("users/cnav/Limites_Andalucia")
pisos = ee.FeatureCollection("users/cnav/PisosBioclim_10_SN")

# Based on Rivas-Martinez https:#biogeografia.net/bioclima07a.html
Map.addLayer(Andalucia, {}, "Andalucia")

# Based on Rivas-Martinez https:#biogeografia.net/bioclima07a.html

#********** TIME INTERVAL **********#
startyear = 2010
endyear = 2015
data_inicial = ee.Date.fromYMD(startyear,1,1)
data_final = ee.Date.fromYMD(endyear,12,31)
years = ee.List.sequence(startyear,endyear)

##################/
####### Variables######
#################/
temperature=TempMean \
.filterDate(data_inicial, data_final);

#Calculate an average temperature value for the selected period
temp_anual_mean=temperature.mean()

#Add the layer to the map
Map.addLayer(temp_anual_mean, {'min':13, 'max':16}, "Annual mean temperature", 0)

#Select the coldest month and then calculate the average minimum temperature value
Enero=TempMin \
.filter(ee.Filter.calendarRange(1,1, 'month')) \
.filterDate(data_inicial,data_final)

#print("Enero min:", Enero.size().getInfo())

#Select the coldest month and then calculate the average maximum temperature value
Enero_max = TempMax \
.filter(ee.Filter.calendarRange(1,1, 'month')) \
.filterDate(data_inicial,data_final)

#print("Enero max:", Enero_max.size().getInfo())


#################################/
####### Calculation of the "Thermicity" index#######/
#################################/
#Indice de Termicidad
#It: Indice de termicidad de Rivas-Martinez [It = (T + m + Mn) * 10]

# Respuesta rapida min y max promedio
MeanMin = Enero.mean().toInt32()
Map.addLayer(MeanMin, {}, "Promedio de las temperaturas mas bajas", 0)
#print("MeanMIN:", MeanMin.getInfo())
MeanMax = Enero_max.mean().toInt32().select("b1")
#print("MeanMAX:", MeanMax.getInfo())

#print("temp_anual_mean:",temp_anual_mean.getInfo())

IT= (temp_anual_mean).add(MeanMin).add(MeanMax)
IT=IT.multiply(10)
#print("Indice de termicidad:", IT.getInfo())

Map.addLayer(IT, {'min':169, 'max':301}, "Indice de termicidad")


###########################
#######/PISO CRIOROMEDITERRANEO########
###########################
# I create criteria that it must meet and then I consider it as belonging to the apartment if it meets at least 2 of the criteria, giving greater weight to the thermal index

cri = IT.lt(-30).multiply(2)

max_temp_mask = MeanMax.lt(0)
min_temp_mask = MeanMin.lt(-7)
mean_temp_mask = temp_anual_mean.lt(4)
masks_sum = cri.add(max_temp_mask).add(min_temp_mask).add(mean_temp_mask)
mask_res = masks_sum.gte(3)
crioromediterraneo = IT.updateMask(cri)

# Assign a specific value to the Piso bioclimatico
crioromediterraneo = crioromediterraneo.where(crioromediterraneo, 1).rename("piso")

crioro = pisos.filter(ee.Filter.eq('PISO', "Crioromediterráneo"))
Map.addLayer(crioro, {}, "Piso CriOromediterráneo fijo", 0)
Map.addLayer(crioromediterraneo, {'palette':["purple"]}, "Piso CriOromediterraneo raster", 0)

###########################
#######/ OROMEDITERRANEO###########/
###########################

# #.And(MeanMax.gt(0)).And(MeanMax.lt(2)).And(MeanMin.gt(-7)).And(MeanMin.lt(-4)).And(temp_anual_mean.gt(4)).And(temp_anual_mean.lt(8))
#
orom = IT.gt(-30).And(IT.lt(60)).multiply(2)
max_temp_mask = MeanMax.gt(0).And(MeanMax.lt(2))
min_temp_mask = MeanMin.gt(-7).And(MeanMin.lt(-4))
mean_temp_mask = temp_anual_mean.gt(4).And(temp_anual_mean.lt(8))
masks_sum = orom.add(max_temp_mask).add(min_temp_mask).add(mean_temp_mask)
mask_res = masks_sum.gte(3)
oromediterraneo = IT.updateMask(mask_res)

# Assign a specific value to the Piso bioclimatico
oromediterraneo = oromediterraneo.where(oromediterraneo, 4).rename("piso")
#To compare, load the corresponding piso of the REDIAM shape
oro = pisos.filter(ee.Filter.eq('PISO', "Oromediterráneo"))
Map.addLayer(oro, {}, "Piso Oromediterraneo fijo",0)
Map.addLayer(oromediterraneo, {'palette':["blue"]}, "Piso Oromediterraneo raster",0)

###########################
#######/ SUPRAMEDITERRANEO##########/
###########################

sup = IT.gt(60).And(IT.lt(210)).multiply(2)
#.And(MeanMax.gt(2)).And(MeanMax.lt(9)).And(MeanMin.gt(-4)).And(MeanMin.lt(1)).And(temp_anual_mean.gt(8)).And(temp_anual_mean.lt(13))
max_temp_mask = MeanMax.gt(2).And(MeanMax.lt(9))
min_temp_mask = MeanMin.gt(-4).And(MeanMin.lt(1))
mean_temp_mask = temp_anual_mean.gt(8).And(temp_anual_mean.lt(13))
masks_sum = sup.add(max_temp_mask).add(min_temp_mask).add(mean_temp_mask)
mask_res = masks_sum.gte(3)
supramediterraneo = IT.updateMask(mask_res)

# Assign a specific value to the Piso bioclimatico
supramediterraneo = supramediterraneo.where(supramediterraneo, 7).rename("piso")

#To compare, load the corresponding piso of the REDIAM shape
supra = pisos.filter(ee.Filter.eq('PISO', "Supramediterráneo"))
Map.addLayer(supra, {}, "Piso Supramediterraneo fijo",0)
Map.addLayer(supramediterraneo, {'palette':["green"]}, "Piso Supramediterraneo raster", 0)

###########################
#######/ MESOMEDITERRANEO###########
###########################

mes = IT.gt(210).And(IT.lt(350)).multiply(2)
#.And(MeanMax.gt(9)).And(MeanMax.lt(14)).And(MeanMin.gt(1)).And(MeanMin.lt(4)).And(temp_anual_mean.gt(13)).And(temp_anual_mean.lt(17))
max_temp_mask = MeanMax.gt(9).And(MeanMax.lt(14))
min_temp_mask = MeanMin.gt(1).And(MeanMin.lt(4))
mean_temp_mask = temp_anual_mean.gt(13).And(temp_anual_mean.lt(17))
masks_sum = mes.add(max_temp_mask).add(min_temp_mask).add(mean_temp_mask)
mask_res = masks_sum.gte(2)
mesomediterraneo = IT.updateMask(mask_res)

# Assign a specific value to the Piso bioclimatico

mesomediterraneo = mesomediterraneo.where(mesomediterraneo, 11).rename("piso")
#To compare, load the corresponding piso of the REDIAM shape
meso = pisos.filter(ee.Filter.eq('PISO', "Mesomediterráneo"))
Map.addLayer(meso, {}, "Piso Mesomediterraneo fijo",0)
Map.addLayer(mesomediterraneo, {'palette':["red"]}, "Piso Mesomediterráneo raster", 0)

###########################
#######/ TERMOMEDITERRANEO##########/
###########################

ter = IT.gt(350).And(IT.lt(470)).multiply(2)
#.And(MeanMax.gt(14)).And(MeanMax.lt(18)).And(MeanMin.gt(4)).And(MeanMin.lt(10)).And(temp_anual_mean.gt(17)).And(temp_anual_mean.lt(19))
max_temp_mask = MeanMax.gt(14).And(MeanMax.lt(18))
min_temp_mask = MeanMin.gt(4).And(MeanMin.lt(10))
mean_temp_mask = temp_anual_mean.gt(17).And(temp_anual_mean.lt(19))
masks_sum = ter.add(max_temp_mask).add(min_temp_mask).add(mean_temp_mask)
mask_res = masks_sum.gte(3)
termomediterraneo = IT.updateMask(mask_res)

#  Assign a specific value to the Piso bioclimatico
termomediterraneo = termomediterraneo.where(termomediterraneo, 15).rename("piso")
#To compare, load the corresponding piso of the REDIAM shape
termo = pisos.filter(ee.Filter.eq('PISO', "Termomediterráneo"))
Map.addLayer(termo, {}, "Piso Termomediterraneo fijo",0)
Map.addLayer(termomediterraneo, {'palette':["darkred"]}, "Piso Termomediterráneo raster", 0)

##########################################/
##Join all##############################
##########################################

pisos_bioclimaticos = ee.ImageCollection([crioromediterraneo, oromediterraneo, supramediterraneo,mesomediterraneo,termomediterraneo]).mosaic()

Map.addLayer(pisos_bioclimaticos, {'min':1, 'max':15, 'palette':["ff450c","ffee0c","0fc036","0237ff","c602c8"]}, "Pisos joined")

Map.centerObject(Andalucia, 8)
Map

Map(center=[37.46773396385332, -4.577988174242144], controls=(WidgetControl(options=['position', 'transparent_…

# A continuacion el código para descargar las imagenes 
## primero puedo querer dibujar un area de interes para descargar 


In [16]:
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature

if feature is None:
    geom = ee.Geometry.Polygon(
        [
            [
                [-3.726, 37.222],
                [-3.726, 36.915],
                [-2.479, 36.915],
                [-2.479, 37.222],
                [-3.726, 37.222],
            ]
        ]
    )
    feature = ee.Feature(geom, {})

roi = feature.geometry()

Luego lo pongo en el mapa 

In [17]:
Map = geemap.Map()
Map.addLayer(roi)
Map



Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Y ahora sí configuro la carpeta de descarga 
import ee
import geemap
import os

In [19]:
import ee
import geemap
import os

In [23]:
## Defino la carpeta de salida 

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'pisos_bioclimaticos.tif')

In [24]:
image = pisos_bioclimaticos.clip(roi).unmask()

geemap.ee_export_image(
    image, filename=filename, scale=100, region=roi, file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\carlo\Downloads\pisos_bioclimaticos.tif


In [ ]:
## Si quisiera descargarlas en Gdrive

In [25]:
geemap.ee_export_image_to_drive(
    image, description='PisosBioclimaticos', folder='export', region=roi, scale=100
)